# How should the hidden states: vel & acc be treated?

## Purpose
* The velocity and accelerations are not measured in model test usually they are "hidden states" that needs to be estimated for the regression

## Methodology
* Load simulated data generated by: [12.05_regression_simulated_data_simple_nonlinear.ipynb](12.05_regression_simulated_data_simple_nonlinear.ipynb)
* Determine velocity and acceleration and compared...

## Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_simple_nonlinear  as vmm
from vessel_manoeuvring_models.data.case_1 import ship_parameters, df_parameters, ps, ship_parameters_prime
from vessel_manoeuvring_models.data.transform import transform_to_ship

## Ship parameters

In [ ]:
ship_parameters

## Brix parameters

In [ ]:
mask = df_parameters['prime'].notnull()
index = df_parameters.loc[mask,'prime'].index
coefficients=vmm.simulator.get_all_coefficients(sympy_symbols=False)
missing_coefficients = set(coefficients) - set(index)
missing_coefficients

In [ ]:
mask = df_parameters['prime'].notnull()
df_parameters.loc[mask,'prime']

## Load simulate data

In [ ]:
df_result = pd.read_csv('../data/processed/simple_simulation.csv', index_col=0)
df_result['z0']=0
df_measurement = df_result.drop(columns=['u','v','r','u1d','v1d','r1d'])  # Removing vel and acc

In [ ]:
ship_parameters['x_G']

### Check accelerations

In [ ]:
import scipy.integrate

In [ ]:
u_integrated = df_result.iloc[0]['u'] + scipy.integrate.cumtrapz(y=df_result['u1d'], 
                                                                 x=df_result.index)
fig,ax=plt.subplots()
df_result.plot(y='u', ax=ax)
ax.plot(df_result.index[1:], u_integrated, '--', label='u_integrated')
ax.legend();

### Check transformation

In [ ]:
t_ = df_measurement.index

suffix = ['','1d','2d']
for i in range(2):
    for key in ['x0','y0','z0','psi']:
        df_measurement[f'{key}{suffix[i+1]}'] = np.gradient(df_measurement[f'{key}{suffix[i]}'], t_) 
        
df_measurement = transform_to_ship(df=df_measurement)
df_measurement=df_measurement.iloc[2:-2].copy()

In [ ]:
from pykalman import KalmanFilter

def filter(df, key='x0', observation_covariance = 100000):
    
    df = df.copy()
    t = df.index
    dt = t[1] - t[0]
    
    A = np.array([[1, dt, 0.5*(dt**2)],
                  [0,  1, dt         ],
                  [0,  0, 1          ],
                    ])
    
    kf = KalmanFilter(transition_matrices=A,
                      initial_state_mean = [df[key].iloc[0:100].median(),df[f'{key}1d'].iloc[0:100].median(),df[f'{key}2d'].iloc[0:100].median()],
                      random_state=np.random.RandomState(0),
                      transition_covariance=100 * np.eye(3),
                      observation_covariance=observation_covariance * np.eye(1),
    
                      em_vars=[
                          'transition_covariance', 
                          'observation_covariance', 
                          'initial_state_mean', 
                          'initial_state_covariance'
                          ],
    
                      )
    
    observations = df[key]
    states_pred = kf.em(observations).smooth(observations)[0]
    
    df[f'{key}'] = states_pred[:,0]
    df[f'{key}1d'] = states_pred[:,1]
    df[f'{key}2d'] = states_pred[:,2]
    
    return df

In [ ]:
df_measurement_filtered = df_measurement.copy()

df_measurement_filtered = filter(df=df_measurement_filtered, key='x0', observation_covariance = 100000)
df_measurement_filtered = filter(df=df_measurement_filtered, key='y0', observation_covariance = 100000)
df_measurement_filtered = filter(df=df_measurement_filtered, key='psi', observation_covariance = 100000)

df_measurement_filtered.head()

### Check Kalman filter

In [ ]:
for key in ['x0', 'y0', 'z0', 'x01d', 'x02d', 'y01d', 'y02d', 'psi1d','psi2d']:
    fig,ax=plt.subplots()
    fig.set_size_inches(15,1)
    
    df_measurement.plot(y=key, ax=ax, label='simulation')
    df_measurement_filtered.plot(y=key, ax=ax, label='kalman filtered')
    ax.set_ylabel(key)

In [ ]:
for i in range(2):
    for key in ['u','v','r']:
        y = f'{key}{suffix[i]}'
        
        fig,ax=plt.subplots()
        df_result.plot(y=y,label='sim', ax=ax)
        df_measurement.plot(y=y,label='measurement', style='--', ax=ax)
        ax.set_ylabel(y)